# Overview

This notebook introduces the plotly.py `FigureWidget` that was introduced in version 3

### Imports

In [1]:
# pandas
import pandas as pd

# numpy
import numpy as np

### Load iris dataset

In [2]:
iris_df = pd.read_csv('datasets/iris.csv')

## Create and display an empty FigureWidget
A FigureWidget behaves almost identically to a Figure but it is also an ipywidget that can be displayed directly in the notebook without calling `iplot`

In [3]:
import plotly.graph_objs as go

In [4]:
f1 = go.FigureWidget()
f1

FigureWidget({
    'data': [], 'layout': {}
})

# Tab completion 
Entering ``f1.add_<tab>`` displays add methods for all of the supported trace types

In [5]:
# f1.add_scatter

Entering ``f1.add_scatter(<tab>)`` displays the names of all of the top-level properties for the scatter trace type

Entering ``f1.add_scatter(<shift+tab>)`` displays the signature pop-up. Expanding this pop-up reveals the method doc string which contains the descriptions of all of the top level properties

In [6]:
# f1.add_scatter(

# Add scatter trace

In [7]:
scatt1 = f1.add_scatter(x=iris_df.sepal_length, y=iris_df.petal_width)

In [8]:
# That's not what we wanted, change the mode to 'markers'
scatt1.mode = 'markers'

In [9]:
# Set size to 8
scatt1.marker.size = 8

In [10]:
# Add axis labels
f1.layout.xaxis.title = 'sepal_length'
f1.layout.yaxis.title = 'petal_width'

In [11]:
# Hover info
scatt1.text = iris_df.species
scatt1.hoverinfo = 'text+x+y'
f1.layout.hovermode = 'closest'

## Animate marker size change

In [12]:
# Set marker size based on petal_length
with f1.batch_animate(duration=1000):
    scatt1.marker.size = np.sqrt(iris_df.petal_length.values * 50)
    

In [14]:
# Restore constant marker size
with f1.batch_animate(duration=1000):
    scatt1.marker.size = 8

## Configure colorscale for brushing

In [15]:
scatt1.marker.colorbar = None
scatt1.marker.colorscale = [[0, 'lightgray'], [0.5, 'lightgray'], [0.5, 'red'], [1, 'red']]
scatt1.marker.cmin = -0.5
scatt1.marker.cmax = 1.5
scatt1.marker.colorbar.ticks = 'outside'
scatt1.marker.colorbar.tickvals = [0, 1]
scatt1.marker.colorbar.ticktext = ['unselected', 'selected']

In [16]:
# Reset colors to zeros (unselected)
scatt1.marker.color = np.zeros(len(iris_df))
selected = np.zeros(len(iris_df))

### Configure brushing callback

In [17]:
# Assigning these variables here is not required. But doing so tricks Jupyter into 
# providing property tab completion on the parameters to the brush function below
from plotly.callbacks import Points
trace, points = scatt1, Points()

In [18]:
def brush(trace, points, selector):
    inds = np.array(points.point_inds)
    if inds.size:
        selected[inds] = 1
        trace.marker.color = selected

In [19]:
scatt1.on_selection(brush)

Now box or lasso select points on the figure and see them turn red

In [20]:
# Reset brush
selected = np.zeros(len(iris_df))
scatt1.marker.color = selected

## Create second plot with different features

In [21]:
f2 = go.FigureWidget(data=[
    go.Scatter(x=iris_df.petal_length, y=iris_df.sepal_width, mode='markers')])
f2

FigureWidget({
    'data': [{'mode': 'markers',
              'type': 'scatter',
              'uid': '83da315…

In [22]:
# Set axis titles
f2.layout.xaxis.title = 'petal_length'
f2.layout.yaxis.title = 'sepal_width'

In [23]:
# Grab trace reference
scatt2 = f2.data[0]

In [24]:
# Set marker styles / colorbars to match between figures
scatt2.marker = scatt1.marker

In [25]:
# Configure brush on both plots to update both plots
def brush(trace, points, state):
    inds = np.array(points.point_inds)
    if inds.size:
        selected = scatt1.marker.color.copy()
        selected[inds] = 1
        scatt1.marker.color = selected
        scatt2.marker.color = selected    
    
scatt1.on_selection(brush)
scatt2.on_selection(brush)

In [26]:
# Reset brush
def reset_brush(btn):
    selected = np.zeros(len(iris_df))
    scatt1.marker.color = selected
    scatt2.marker.color = selected

In [27]:
# ipywidgets
from ipywidgets import HBox, VBox, Button

In [28]:
# Create reset button
button = Button(description="clear")
button.on_click(reset_brush)
button

Button(description='clear', style=ButtonStyle())

In [29]:
# Hide colorbar for figure 1
scatt1.marker.showscale = False

In [30]:
# Set dragmode to lasso for both plots
f1.layout.dragmode = 'lasso'
f2.layout.dragmode = 'lasso'

In [31]:
# Display two figures and the reset button
f1.layout.width = 500
f2.layout.width = 500

In [32]:
VBox([HBox([f1, f2]), button])

    'data': [{'hoverinfo': 'text+x+y',
              'marker': {'…